### Naver movie 데이터 세트

* 데이터 전처리
* 임베딩(Word2Vec)
* 모델 : Logistic Regression

In [1]:
import os
import logging

import pandas as pd
import numpy as np

from gensim.models import word2vec # word2vec 사용
from sklearn.model_selection import train_test_split # 데이터 셋 분할
from sklearn.linear_model import LogisticRegression # 모델 회귀 모형 이용

### [1] 데이터 세트 불러오기

In [2]:
BASIC_PATH = "/dataset"

In [3]:
DATA_CLEAN_DATA = BASIC_PATH + "/train_clean.csv"

In [4]:
train_data = pd.read_csv(DATA_CLEAN_DATA)

In [5]:
print(train_data.isnull().values.any()) # Null 값이 존재하는지 확인

False


In [6]:
documents = list(train_data['document'])
label = list(train_data['label'])

In [7]:
sentences = []
for s in documents:
    sentences.append(s.split())

In [8]:
print(sentences[:5])

[['더빙', '진짜', '짜증나다', '목소리'], ['흠', '포스터', '보고', '초딩', '영화', '줄', '오버', '연기', '조차', '가볍다', '않다'], ['너', '무재', '밓었', '다그', '래서', '보다', '추천', '다'], ['교도소', '이야기', '구먼', '솔직하다', '재미', '없다', '평점', '조정'], ['사이', '몬페', '그', '익살스럽다', '연기', '돋보이다', '영화', '스파이더맨', '에서', '늙다', '보이다', '하다', '커스틴', '던스트', '너무나도', '이쁘다', '보이다']]


### [2] 임베딩(word2vec)

In [19]:
# 학습 시 필요한 하이퍼 파라미터

num_features = 100 # 워드 백터 특정값 수 -> 임베딩 차원 수 지정
min_word_count = 40 # 단어에 대한 최소 빈도 수 -> 빈도수가 적은 단어들은 학습 x
num_workers = 4 # 프로세스 개수 
context = 10 # 컨텍스트 윈도우 크기
downsampling = 1e-3 # 다운 샘플링 비율

In [20]:
logging.basicConfig(format='%(astime)s : %(levelname)s : %(messages)s', level=logging.INFO)

In [21]:
model = word2vec.Word2Vec(sentences, workers=num_workers, \
                         size=num_features, min_count = min_word_count, \
                         window = context, sample = downsampling)

--- Logging error ---
Traceback (most recent call last):
  File "c:\users\msi\appdata\local\programs\python\python37\lib\logging\__init__.py", line 1034, in emit
    msg = self.format(record)
  File "c:\users\msi\appdata\local\programs\python\python37\lib\logging\__init__.py", line 880, in format
    return fmt.format(record)
  File "c:\users\msi\appdata\local\programs\python\python37\lib\logging\__init__.py", line 622, in format
    s = self.formatMessage(record)
  File "c:\users\msi\appdata\local\programs\python\python37\lib\logging\__init__.py", line 591, in formatMessage
    return self._style.format(record)
  File "c:\users\msi\appdata\local\programs\python\python37\lib\logging\__init__.py", line 433, in format
    return self._fmt % record.__dict__
KeyError: 'astime'
Call stack:
  File "c:\users\msi\appdata\local\programs\python\python37\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "c:\users\msi\appdata\local\programs\python\python37\lib\runpy.

In [22]:
def get_feature(words, model, num_feature):
    # 우리는 num_feature 차원의 벡터 크기를 만들거임
    feature_vector = np.zeros((num_features), dtype=np.float32) # feature_vector 초기화
    
    num_words = 0
    
    # index2word_set -> 문장 속 단어가 모델 단어 사전에 있는지 보기 위해 어휘 사전 부름
    index2word_set = set(model.wv.index2word) # 어휘 사전 준비
    
    for w in words:
        if w in index2word_set: # 단어가 모델 단어 사전에 있으면
            num_words += 1
            feature_vector = np.add(feature_vector, model.wv[w]) #wv[w] -> 해당 단어의 임베딩 벡터 가져와 모든 벡터들의 합을 구함
    
    feature_vector = np.divide(feature_vector, num_words) # 평균을 내기 위해서 feature vector / num_words(단어 사전에 있는 단어들)한다
        
    return feature_vector

In [23]:
def get_dataset(sentences, model, num_features):
    dataset = list()
    not_dataset = list()
    
    for idx, s in enumerate(sentences): # 한 문서씩 꺼냄 (단어 배열 형태 임)
        result = get_feature(s, model, num_features)
        if np.isnan(result).any():
            #print("check -> nan => ", words)
            not_dataset.append(idx)
        else:
            dataset.append(result) # 이 단어 배열들을 임베딩 함
    
    reviewFeatureVecs = np.stack(dataset)
    return reviewFeatureVecs, not_dataset

In [24]:
# X 벡터화-임베딩ㅇ 진행 -> 실제 학습에 사용될 입력값을 만들어 보자.
X, not_X = get_dataset(sentences, model, num_features)

c:\users\msi\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in true_divide
  from ipykernel import kernelapp as app


In [28]:
# NAN 제거
y = np.array([l for l in range(len(label)) if l not in not_X])

In [29]:
print(X.shape)
print(y.shape)

(147668, 100)
(147668,)


### [3] 데이터 세트 분할 (Train + Validation)

In [30]:
RANDOM_SEED = 42
TEST_SPLIT = 0.2

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=TEST_SPLIT, random_state=RANDOM_SEED)

### [4] 모델 선언 및 훈련

In [31]:
lgs = LogisticRegression(class_weight='balanced')
lgs.fit(X_train, y_train)

MemoryError: Unable to allocate 104. GiB for an array with shape (118134, 118134) and data type int64

### [5] 모델 예측(정확도 측정)

In [ ]:
predicted = lgs.predict(X_val)

In [ ]:
print("Accuracy : %f" % lgs.score(X_val, y_val)) # score()함수 => 정확도 측정

### [6] TEST 데이터 세트로 모델 확인(예측)

In [ ]:
TEST_CLEAN_DATA = BASIC_PATH + "/test_clean.csv"
test_data = pd.read_csv(TEST_CLEAN_DATA)

In [ ]:
test_data.head(5)

In [ ]:
test_review = list(test_data['document'])

test_sentences = list()

for review in test_review:
    test_sentences.append(review.split())

In [ ]:
print(len(test_sentences))

In [ ]:
testDataVecs = get_dataset(test_sentences, model, num_features)

In [ ]:
test_predicted = lgs.predict(testDataVecs)

In [ ]:
print(test_predicted)